In [ ]:
import zipfile
import cv2
import pandas as pd
import numpy as np
import os
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense
from joblib import dump, load

In [ ]:
# Google Colab specific steps
from google.colab import files, drive
files.upload()
drive.mount('/content/drive')

In [ ]:
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d adikurniawan/color-dataset-for-color-recognition/
with zipfile.ZipFile('color-dataset-for-color-recognition.zip', 'r') as zip_ref:
    zip_ref.extractall('/kaggle/input/color-dataset-for-color-recognition/')

In [ ]:
def augment_image(image):
    flipped_horizontally = cv2.flip(image, 1)
    return [image, flipped_horizontally]

rgb, hsv, color = [], [], []

In [ ]:
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        image = cv2.imread(os.path.join(dirname, filename))
        image_rgb, image_hsv = cv2.cvtColor(image, cv2.COLOR_BGR2RGB), cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

        for img in augment_image(image_rgb):
            rgb.append(img[0][0])
            color.append(dirname.split('/')[-1])

        for img in augment_image(image_hsv):
            hsv.append(img[0][0])

rgb_df, hsv_df, color_df = pd.DataFrame(np.array(rgb), columns=['red', 'green', 'blue']), \
                           pd.DataFrame(np.array(hsv), columns=['hue', 'saturation', 'value']), \
                           pd.DataFrame({'color': color})

df = pd.concat([rgb_df, hsv_df, color_df], axis=1)
X_train, X_test, y_train, y_test = train_test_split(df.iloc[:, :-1], df.iloc[:, -1], test_size=0.3, random_state=0)

def build_knn_model(input_dim, output_dim):
    model = Sequential()
    model.add(Dense(output_dim, input_dim=input_dim, activation='softmax'))
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

def train_knn_model(X, y):
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    y_encoded = pd.get_dummies(y)

    input_dim, output_dim = X_scaled.shape[1], y_encoded.shape[1]
    model = build_knn_model(input_dim, output_dim)

    model.fit(X_scaled, y_encoded, epochs=10000, batch_size=32, validation_split=0.2)

    return model, scaler

knn_model, scaler = train_knn_model(X_train, y_train)
dump(knn_model, 'knn_model.h5')
dump(scaler, 'scaler.joblib')

loaded_knn_model = load_model('knn_model.h5')
loaded_scaler = load('scaler.joblib')

y_pred = np.argmax(loaded_knn_model.predict(loaded_scaler.transform(X_test)), axis=1)
accuracy = accuracy_score(y_test, y_pred)
print(f'Loaded Model Accuracy: {accuracy}')